# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [32]:
# Write your code below.
from dotenv import load_dotenv
load_dotenv()

True

In [33]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [34]:
import os
from glob import glob
import sys

# Error handling if Env variable doesnt work or any Path issue
try:
    stock_module_path = os.getenv('PRICE_DATA')
    print(stock_module_path)
    sys.path.append(stock_module_path)
    path =sys.path
    print("sys path",path)
    stock_files_path = f'{path[-1]}**/*.parquet'
    print('stock files path',stock_files_path)
    stock_files = glob(stock_files_path,recursive=True)
    if len(stock_files)==0:
        raise Exception('Empty files due to path issue')
    print('stock files***',stock_files)
except Exception:
    stock_files = glob('../../05_src/data/prices/**/*.parquet',recursive=True)
    print('Stock files read directly from PRICE_DATA folder',stock_files)

../../05_src/data/prices/
sys path ['C:\\PythonTraining\\PythonPackages\\KVR1', 'C:\\PythonTraining\\PythonModules', 'c:\\DataScienceInstitue\\production\\02_activities\\assignments', 'c:\\Users\\nages\\miniconda3\\envs\\dsi_participant\\python39.zip', 'c:\\Users\\nages\\miniconda3\\envs\\dsi_participant\\DLLs', 'c:\\Users\\nages\\miniconda3\\envs\\dsi_participant\\lib', 'c:\\Users\\nages\\miniconda3\\envs\\dsi_participant', '', 'c:\\Users\\nages\\miniconda3\\envs\\dsi_participant\\lib\\site-packages', 'c:\\Users\\nages\\miniconda3\\envs\\dsi_participant\\lib\\site-packages\\win32', 'c:\\Users\\nages\\miniconda3\\envs\\dsi_participant\\lib\\site-packages\\win32\\lib', 'c:\\Users\\nages\\miniconda3\\envs\\dsi_participant\\lib\\site-packages\\Pythonwin', 'c:\\Users\\nages\\miniconda3\\envs\\dsi_participant\\lib\\site-packages\\setuptools\\_vendor', '../../05_src/data/prices/', '../../05_src/data/prices/', '../../05_src/data/prices/', '../../05_src/data/prices/', '../../05_src/data/prices

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [35]:
print('stock file',len(stock_files))
parquet_df = dd.read_parquet(stock_files).set_index('ticker')
parquet_df.columns
dd_shift = parquet_df.groupby('ticker', group_keys=False).apply(lambda x: x.assign(Close_lag_1 = x['Close'].shift(1),Adj_Close_1 = x['Adj Close'].shift(1)))
dd_shift.compute().head()


stock file 2045


C:\Users\nages\AppData\Local\Temp\ipykernel_42572\2190604269.py:4: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = parquet_df.groupby('ticker', group_keys=False).apply(lambda x: x.assign(Close_lag_1 = x['Close'].shift(1),Adj_Close_1 = x['Adj Close'].shift(1)))


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_1
ticker,,,,,,,,,,,
ACN,2001-07-19,15.10,15.29,15.00,15.17,11.404394,34994300.0,ACN.csv,2001,NaN,NaN
ACN,2001-07-20,15.05,15.05,14.80,15.01,11.284108,9238500.0,ACN.csv,2001,15.17,11.404394
ACN,2001-07-23,15.00,15.01,14.55,15.00,11.276587,7501000.0,ACN.csv,2001,15.01,11.284108
ACN,2001-07-24,14.95,14.97,14.70,14.86,11.171341,3537300.0,ACN.csv,2001,15.00,11.276587
ACN,2001-07-25,14.70,14.95,14.65,14.95,11.238999,4208100.0,ACN.csv,2001,14.86,11.171341


In [36]:
dd_shift['returns'] = dd_shift['Close']/dd_shift['Close_lag_1']
dd_shift['hi_lo_range'] = dd_shift['High'] - dd_shift['Low']
dd_shift.compute().head()
dd_feat = dd_shift
dd_feat.compute().head()

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_1,returns,hi_lo_range
ticker,,,,,,,,,,,,,
ACN,2018-01-02,153.500000,154.100006,152.779999,153.839996,148.655960,3061900.0,ACN.csv,2018,NaN,NaN,NaN,1.320007
ACN,2018-01-03,152.990005,154.990005,152.990005,154.550003,149.342056,2064200.0,ACN.csv,2018,153.839996,148.655960,1.004615,2.000000
ACN,2018-01-04,155.000000,156.860001,154.770004,156.380005,151.110382,1777000.0,ACN.csv,2018,154.550003,149.342056,1.011841,2.089996
ACN,2018-01-05,156.610001,157.720001,156.130005,157.669998,152.356918,1597600.0,ACN.csv,2018,156.380005,151.110382,1.008249,1.589996
ACN,2018-01-08,157.369995,159.009995,156.839996,158.929993,153.574463,2616900.0,ACN.csv,2018,157.669998,152.356918,1.007991,2.169998


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [37]:
# this converts Dask dataframe to pandas data frame
pd_df = dd_feat.compute()
pd_df.groupby('ticker').apply(lambda x: x.assign(Roll_Avg_10 = x['returns'].rolling(10).mean()))
pd_df.head()


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_1,returns,hi_lo_range
ticker,,,,,,,,,,,,,
ACN,2001-07-19,15.10,15.29,15.00,15.17,11.404394,34994300.0,ACN.csv,2001,NaN,NaN,NaN,0.29
ACN,2001-07-20,15.05,15.05,14.80,15.01,11.284108,9238500.0,ACN.csv,2001,15.17,11.404394,0.989453,0.25
ACN,2001-07-23,15.00,15.01,14.55,15.00,11.276587,7501000.0,ACN.csv,2001,15.01,11.284108,0.999334,0.46
ACN,2001-07-24,14.95,14.97,14.70,14.86,11.171341,3537300.0,ACN.csv,2001,15.00,11.276587,0.990667,0.27
ACN,2001-07-25,14.70,14.95,14.65,14.95,11.238999,4208100.0,ACN.csv,2001,14.86,11.171341,1.006057,0.30


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
## Not necessarily as Dask and Pandas are similar in terms of dataframe operations with Dask is
## lazily executed. If data is too small which seems to be in this case, It's Ok to conver to
## pandas data frame and use it accordingly
+ Would it have been better to do it in Dask? Why?
## I would perfrom operations using Dask since Dask only executed the operation only when executed or perform compute() operation and it uses machines  on computer for parallelism using scheduler.
## with Dask dataframe, we can use lazy evaluation implicitly and split the data
## into multiple chunks and have parallel processing using multiprocessing capabilities of dask

## Panadas, Dask, SQL are data processing tools/choices to process data with each offering certain advantages over other ones.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.